In [1]:
import os

In [2]:
%pwd

'd:\\MyCase\\Projects\\DSAI\\portfolio\\Text-Summarization-NLP\\research'

In [3]:
os.chdir("D:\MyCase\Projects\DSAI\portfolio\Text-Summarization-NLP")

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\rosha\AppData\Local\Temp\ipykernel_29132\1817716843.py:1: SyntaxWarning: invalid escape sequence '\M'
  os.chdir("D:\MyCase\Projects\DSAI\portfolio\Text-Summarization-NLP")


In [4]:
%pwd

'D:\\MyCase\\Projects\\DSAI\\portfolio\\Text-Summarization-NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path
import requests

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from urllib import request, error

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        dst = Path(self.config.local_data_file)
        dst.parent.mkdir(parents=True, exist_ok=True)

        url = self.config.source_URL
        resp = requests.get(url, stream=True, allow_redirects=True, timeout=60)
        print("status:", resp.status_code)
        print("headers:", resp.headers)

        resp.raise_for_status()  # raise if 4xx/5xx

        with dst.open("wb") as f:
            for chunk in resp.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        size = dst.stat().st_size
        print("written bytes:", size)
        if size == 0:
            raise RuntimeError("Downloaded file is 0 bytes, aborting.")

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        zip_path = Path(self.config.local_data_file)
        unzip_path = Path(self.config.unzip_dir)
        unzip_path.mkdir(parents=True, exist_ok=True)

        try:
            with zipfile.ZipFile(str(zip_path), "r") as zip_ref:
                logger.info("Zip contents: %s", zip_ref.namelist()[:10])
                zip_ref.extractall(str(unzip_path))
        except (zipfile.BadZipFile, EOFError) as e:
            sample = zip_path.read_bytes()[:1024].decode(errors="replace")
            raise RuntimeError(
                f"Zip extraction failed (corrupt/truncated). Size={zip_path.stat().st_size}. Preview:\n{sample}"
            ) from e


In [10]:
# # ...existing code...
# import requests
# import shutil
# import zipfile
# from pathlib import Path

# class DataIngestion:
#     def __init__(self, config: DataIngestionConfig):
#         self.config = config

#     def download_file(self):
#         dst = Path(self.config.local_data_file)
#         dst.parent.mkdir(parents=True, exist_ok=True)

#         # atomic download to temporary file then rename
#         tmp = dst.with_suffix(".tmp")
#         url = str(self.config.source_URL)
#         logger.info(f"Downloading from: {url} -> {dst}")

#         try:
#             with requests.get(url, stream=True, timeout=120) as r:
#                 r.raise_for_status()
#                 with tmp.open("wb") as f:
#                     for chunk in r.iter_content(chunk_size=8192):
#                         if chunk:
#                             f.write(chunk)
#                     f.flush()
#                     os.fsync(f.fileno())
#             tmp.replace(dst)  # atomic on same filesystem
#         except Exception as e:
#             if tmp.exists():
#                 tmp.unlink()
#             raise RuntimeError(f"Download failed for {url}: {e}") from e

#         size = dst.stat().st_size
#         logger.info(f"Downloaded {dst} ({size} bytes)")

#         # sanity check: must start with PK
#         with dst.open("rb") as f:
#             head = f.read(8)
#         if not head.startswith(b"PK"):
#             preview = dst.read_bytes()[:512].decode(errors="replace")
#             raise ValueError(f"Downloaded file does not look like a ZIP. Preview:\n{preview}")

#     def extract_zip_file(self):
#         zip_path = Path(self.config.local_data_file).resolve()
#         unzip_path = Path(self.config.unzip_dir).resolve()
#         unzip_path.mkdir(parents=True, exist_ok=True)

#         if not zip_path.exists():
#             raise FileNotFoundError(f"Zip file not found: {zip_path}")
#         if zip_path.stat().st_size == 0:
#             raise ValueError(f"Zip file is empty: {zip_path}")

#         # top-level magic check
#         with zip_path.open("rb") as f:
#             head = f.read(4)
#         if not head.startswith(b"PK"):
#             preview = zip_path.read_bytes()[:512].decode(errors="replace")
#             raise RuntimeError(f"Top-level file is not a ZIP. Preview:\n{preview}")

#         # extract top-level zip
#         try:
#             with zipfile.ZipFile(str(zip_path), "r") as zip_ref:
#                 logger.info("Top-level zip contents: %s", zip_ref.namelist()[:30])
#                 zip_ref.extractall(str(unzip_path))
#         except (zipfile.BadZipFile, EOFError) as e:
#             sample = zip_path.read_bytes()[:1024].decode(errors="replace")
#             raise RuntimeError(f"Failed to extract top-level zip. Size={zip_path.stat().st_size}. Preview:\n{sample}") from e

#         # If the archive contains an inner 'data.zip', extract it properly
#         inner = unzip_path / "data.zip"
#         if inner.exists():
#             if inner.stat().st_size == 0:
#                 raise RuntimeError(f"Inner data.zip extracted as 0 bytes: {inner}")
#             # verify inner magic bytes
#             with inner.open("rb") as f:
#                 if not f.read(4).startswith(b"PK"):
#                     preview = inner.read_bytes()[:512].decode(errors="replace")
#                     raise RuntimeError(f"Inner file is not a ZIP. Preview:\n{preview}")
#             # extract inner zip into same unzip_path (or a subdir)
#             try:
#                 with zipfile.ZipFile(str(inner), "r") as zf:
#                     logger.info("Inner zip contents: %s", zf.namelist()[:30])
#                     zf.extractall(str(unzip_path))
#             except (zipfile.BadZipFile, EOFError) as e:
#                 sample = inner.read_bytes()[:1024].decode(errors="replace")
#                 raise RuntimeError(f"Failed to extract inner data.zip. Size={inner.stat().st_size}. Preview:\n{sample}") from e

#         # diagnostic: list extracted files and sizes
#         for f in sorted(unzip_path.rglob("*"))[:200]:
#             try:
#                 logger.info("extracted: %s (%d)", f.relative_to(unzip_path), f.stat().st_size)
#             except Exception:
#                 logger.info("extracted: %s (error)", f)
# # ...existing code...

In [11]:
# ...existing code...
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    import traceback, sys
    print("Exception type:", type(e).__name__)
    print("Exception message:", str(e))
    traceback.print_exc()         # full traceback for debugging
    raise                         # preserve original traceback


[2025-11-20 18:57:07,911: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-20 18:57:07,916: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-20 18:57:07,919: INFO: common: created directory at: artifacts]
[2025-11-20 18:57:07,921: INFO: common: created directory at: artifacts/data_ingestion]
status: 200
headers: {'Connection': 'keep-alive', 'Content-Length': '23627009', 'Cache-Control': 'max-age=300', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'; sandbox", 'Content-Type': 'application/zip', 'ETag': 'W/"cf8e6fcd6e30e06d4b81d01e8e8873b531e7cdbc435d0a4cf4ba04fdeb8c461d"', 'Strict-Transport-Security': 'max-age=31536000', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection': '1; mode=block', 'X-GitHub-Request-Id': '1991:952F1:1E9AE5:33B9A3:691F0FBD', 'Accept-Ranges': 'bytes', 'Date': 'Thu, 20 Nov 2025 13:27:09 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-maa10238-MAA', 'X-Cache': 'HIT'